In [ ]:
import random
import os
from litellm import completion
from datasets import load_dataset
import json

In [3]:
ds = load_dataset("nvidia/Nemotron-Personas-USA", split="train", streaming=True)
ds_iter = iter(ds.shuffle(buffer_size=10000))

In [ ]:
MODELS = [
    "openai/gpt-4o",
    "google/gemini-2.0-flash",
]

In [14]:
next(ds_iter)

{'uuid': 'f119c535268d4def8f33cb08578bf75a',
 'professional_persona': 'An experienced field technician named Edward Glass, who blends a practical, hands‑on work ethic with a curiosity for emerging smart‑home and solar technologies, is pursuing HVAC certification and NABCEP credentials while aiming to transition into a crew‑lead role where they can mentor apprentices and streamline workflow efficiency.',
 'sports_persona': 'Edward Glass participates in the Reno Outdoor Club’s seasonal deer‑hunt trips, fishes at Lake Tahoe in summer, hikes the Sierra Nevada trails, follows the Vegas Golden Knights (NHL) and the Reno Aces (AAA baseball), and occasionally joins a local recreational basketball league, balancing rugged individual pursuits with community team spirit in a way they find both grounding and energizing.',
 'arts_persona': 'Edward Glass spends evenings playing electric guitar covers of Jimi Hendrix and Gary Clark Jr. at Reno’s open‑mic nights, captures desert sunsets reminiscent of

In [ ]:
class GeneratorAgent():
    def __init__(self):
        self.persona = next(ds_iter)
        self.system_prompt = f"""
        You have the following persona:
        {json.dumps(self.persona)}

        Come up with 10 new, innovative, realistic business ideas given the prompt.
        """
    def generate(self, prompt):
        response = completion(
        model=random.sample(MODELS, 1)[0],
        messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=1.2
        )
        return response.choices[0].message.content